In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import psycopg2
import seaborn as sns

sns.set_style("whitegrid")
import statsmodels.formula.api as sm
from export_regression_lib import trained_model, odds_ratio, independent_variable_stats
from export_regression_lib import (
    standardized_features,
    simulating_features,
    simulate_continuous_outcomes,
)

# START HERE
## The cell below contains all the mission specific data. Here you will specifity, the independent variables, the dependent variable, and the definitions of positive and negative outcome.

In [ ]:
# CHANGE THESE VALUES
# ONLY USE SINGLE QUESTIONS FOR DEPENDENT AND INDEPENDENT VARIABLES IN FORMAT 'PART_NUM-QUESTION_NUM'
continuous_independent_variables = [
    "Q30", 'Q33', 'Q36', 'Q41'

]
# NOTE MULTIPLE QUESTIONS NEED TO BE CATEGORICAL
categorical_independent_variables = ['Q39', 'Q42']
#categorical_dict = {'Q39': 'Urban', 'Q42': 'Republican',  'Q43': 'Donald Trump'}
categorical_dict = {}
dependent_variable = "Q19_4"
negative_outcomes = ["3","4"]
positive_outcomes = ["1","2"]
grouping = 'rid'

data_file_name = 'data_files/hot_takes_data.csv'
encoded_file_name = 'data_files/hot_takes_encoded.csv'

# After entering the information above, go to Cell -> Run All to see your regression results

In [ ]:
data = pd.read_csv(data_file_name).drop([0,1])
encoded_data = pd.read_csv(encoded_file_name).drop([0,1])
continuous_data = encoded_data[continuous_independent_variables + ['rid', dependent_variable]].dropna()
for cat in categorical_independent_variables:
    data[cat] = data[cat].str.replace("\t"," ")
for cont in continuous_independent_variables + [dependent_variable]:
    continuous_data[cont] = continuous_data[cont].astype(int)

In [ ]:
dep_var = (
    pd.merge(
        data[['rid',dependent_variable]],
        encoded_data[['rid',dependent_variable]],
        on='rid'
    )
    .rename(columns={dependent_variable+'_x': 'Value', dependent_variable+'_y':'Encoding','rid':'count'})
    .groupby(['Value', 'Encoding'])
    .count().reset_index()
)
dep_var

In [ ]:
if categorical_dict:
    pass
else:
    for f in categorical_independent_variables:
        temp_grpd = data[['rid',f]].groupby(f).count().reset_index()
        loc = temp_grpd['rid'].idxmax()
        categorical_dict[f] = temp_grpd[f].iloc[loc]

In [ ]:
def get_outcome(response, positive_outcomes, negative_outcomes):
    if str(response) in positive_outcomes:
        return 1
    elif str(response) in negative_outcomes:
        return 0
    else:
        return None

In [ ]:
dummy_df = pd.DataFrame()
for cat in categorical_independent_variables:
    temp_dummy_df = pd.get_dummies(data[["rid", cat]].dropna(), columns=[cat])
    if len(dummy_df) > 0:
        dummy_df = pd.merge(
            dummy_df,
            temp_dummy_df[
                [
                    col
                    for col in list(temp_dummy_df)
                    if not col.endswith(categorical_dict[cat])
                ]
            ],
            on="rid",
        )
    else:
        dummy_df = temp_dummy_df[
            [
                col
                for col in list(temp_dummy_df)
                if not col.endswith(categorical_dict[cat])
            ]
        ]
continuous_data['outcome'] = encoded_data.apply(
    lambda row: get_outcome(row[dependent_variable], positive_outcomes, negative_outcomes),
    axis=1
)
full_data = pd.merge(
    continuous_data,
    dummy_df,
    on='rid'
)
features = [f for f in list(full_data) if f not in ['rid', 'outcome', dependent_variable]]
categorical_features = [f for f in features if any([f.startswith(cf) for cf in categorical_independent_variables])]

In [ ]:
full_data.corr()

In [ ]:
full_data.groupby('outcome')['rid'].count().reset_index()

In [ ]:
stats = independent_variable_stats(full_data, features)
full_data_std = standardized_features(full_data, continuous_independent_variables, stats)
len_pos = len(full_data_std[full_data_std['outcome'] == 1])
len_neg = len(full_data_std[full_data_std['outcome'] == 0])
sample_size = min(len_pos, len_neg)
training = pd.concat([
    full_data_std[full_data_std['outcome'] == 1].sample(n=sample_size),
    full_data_std[full_data_std['outcome'] == 0].sample(n=sample_size)
])
model = trained_model(training[features], training["outcome"])
model = trained_model(training[features], training["outcome"])

In [ ]:
print(model.summary())
sig_indicies = [i for i,p in enumerate(model.pvalues) if p <= .1]
sig_variables = [
        features[i]
        for i, col in enumerate(features)
        if i in sig_indicies
]
print("")
print("ODDS RATIOS~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(odds_ratio(model, sig_variables, sig_indicies))

In [ ]:
sns.set_style("whitegrid")
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
simulated_features = simulating_features(full_data_std, features)
sig_indicies = [i for i, p in enumerate(model.pvalues) if p <= 0.10]
sig_variables = [
    features[i]
    for i, col in enumerate(features)
    if i in sig_indicies and col not in categorical_features
]
for independent_variable in sig_variables:
    outcome_labels = ["negative", "positive"]
    outcome_choices = [0, 1]
    r_d = simulate_continuous_outcomes(
        simulated_features, model, features, independent_variable
    )
    ax.scatter(
        r_d["response_range"],
        r_d["predictions"],
        label=independent_variable[0:20] + "...",
    )
    ax.set_ylabel("Prediction")
    ax.set_xlabel(independent_variable)
    ax.set_yticks(np.sort(outcome_choices))
    ax.set_yticklabels(
        [
            str(outcome) + " " + choice
            for outcome, choice in zip(outcome_choices, outcome_labels)
        ]
    )
    ax.set_xticks(np.sort(r_d["response_range"]))
    ax.set_xticklabels(
        [
            round(
                stats[independent_variable]["std"]
                * (stats[independent_variable]["mu"] + chc)
            )
            for chc in r_d["question_choices"]
        ],
        rotation=90,
    )
    ax.legend()
    ax.set_ylim(-0.1)
    ax.set_xlim(-0.1)
# plt.savefig("sig_sum_uses.png")
plt.show()

In [ ]:
sns.set_style("whitegrid")
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
simulated_features = simulating_features(full_data_std, features)
sig_indicies = [i for i, p in enumerate(model.pvalues) if p <= 0.10]
sig_variables = [
    features[i]
    for i, col in enumerate(features)
    if i in sig_indicies and col in categorical_features
]
for independent_variable in sig_variables:
    outcome_labels = ["negative", "positive"]
    outcome_choices = [0, 1]
    r_d = simulate_continuous_outcomes(
        simulated_features, model, features, independent_variable
    )
    ax.scatter(
        r_d["response_range"],
        r_d["predictions"],
        label=independent_variable[0:20] + "...",
    )
    ax.set_ylabel("Prediction")
    ax.set_xlabel(independent_variable)
    ax.set_yticks(np.sort(outcome_choices))
    ax.set_yticklabels(
        [
            str(outcome) + " " + choice
            for outcome, choice in zip(outcome_choices, outcome_labels)
        ]
    )
    ax.set_xticks(np.sort(r_d["response_range"]))
    ax.set_xticklabels(
        [
            round(
                stats[independent_variable]["std"]
                * (stats[independent_variable]["mu"] + chc)
            )
            for chc in r_d["question_choices"]
        ],
        rotation=90,
    )
    ax.legend()
    ax.set_ylim(-0.1)
    ax.set_xlim(-0.1)
# plt.savefig("sig_sum_uses.png")
plt.show()

In [ ]:
mod = sm.OLS(full_data_std[[dependent_variable]], full_data_std[features]).fit()
mod = sm.OLS(full_data_std[[dependent_variable]], full_data_std[features]).fit()
print(mod.summary())